In [1]:
!pip install langchain langchain-community langchain-core langchain-google-genai faiss-cpu pypdf

In [2]:
import os
from langchain_google_genai import GoogleGenerativeAIEmbeddings, ChatGoogleGenerativeAI
from langchain_community.vectorstores import FAISS
from langchain_community.document_loaders import PyPDFLoader
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain.chains import ConversationalRetrievalChain
from langchain.schema import SystemMessage

In [4]:
GOOGLE_API_KEY = "Sua_Chave_Aqui"
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY

In [5]:
loader = PyPDFLoader("manual_avancado_nobreak_fxp2000.pdf")
documents = loader.load()

In [7]:
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
vectorstore = FAISS.from_documents(documents, embeddings)
retriever = vectorstore.as_retriever()

In [8]:
chat = ChatGoogleGenerativeAI(model='gemini-2.5-flash', temperature=0)

In [10]:
system_message = SystemMessage(content=('''
Você é um assistente virtual especializado no atendimento ao cliente no nobreak FXP-2000
Responda claramente perguntas técnicas, funcionalidades, garantia, manutenção, atualizações e suporte técnico.
Se a pergunta for irrelevante, responda educadamente recusando a pergunta.
'''))

In [11]:
store = {}
def get_session_history(session_id: str):
  if session_id not in store:
    store[session_id] = InMemoryChatMessageHistory()
  return store[session_id]

qa_chain = ConversationalRetrievalChain.from_llm(
    llm = chat,
    retriever = retriever,
    return_source_documents = False
)

chain_with_history = RunnableWithMessageHistory(
    qa_chain,
    get_session_history,
    input_messages_key="question",
    history_messages_key="chat_history"
)

In [13]:
session_id = "sessao1"
while True:
  pergunta = input("Você: ")
  if pergunta.lower() in ["sair", "quit", "exit"]:
    print("Até logo!")
    break
  resposta = chain_with_history.invoke(
      {"question": pergunta},
      config={"configurable": {"session_id": session_id}}
  )
  print("Assistente:", resposta["answer"])

Você: Qual a autonomia do nobreak?
Assistente: A autonomia típica do nobreak FXP-2000 é de 25 minutos a 50% da carga.
Você: Existe um alarme sonoro de bateria fraca?
Assistente: Sim, alarmes visuais e sonoros são emitidos quando a bateria está fraca.
Você: e de falha interna?
Assistente: Sim, alarmes visuais e sonoros são emitidos em caso de falha interna.
Você: sair
Até logo!
